<a href="https://colab.research.google.com/github/RaiAnant/MangaChroma/blob/master/fasterai/critics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

Wed Aug 14 12:12:49 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    15W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

## Imports

In [0]:
from fastai.core import *
from fastai.torch_core import *
from fastai.vision import *
from fastai.vision.gan import AdaptiveLoss, accuracy_thresh_expand

## Critic Model

In [0]:
# NormType is an enum where Spectral is the 4th element of the enum.
_conv_args = dict(leaky=0.2, norm_type=NormType.Spectral)

In [0]:
# **_conv_args will unwrap the above dict items as args in
def _conv(ni:int, nf:int, ks:int=3,stride:int=1, **kwargs):
  return conv_layer(ni, nf, ks, stride, **_conv_args, **kwargs)

In [0]:
def custom_gan_critic(n_channels:int=3, nf:int=256, n_blocks:int=3, p:int=0.15):
  """Critic for training a GAN"""
  layers = [
            _conv(n_channels, nf, ks=4, stride=2),
            nn.Dropout(p/2)]
  # Deeper layers have more dropout.
  # Probable reason: Deeper layers learn more specialized features while shallower layers learn general features.
  # Thus, deeper layers require more regularisation.
  for i in range(n_blocks):
    layers += [
               _conv(nf, nf, ks=3, stride=1),
               nn.Dropout2d(p),
               _conv(nf, nf*2, ks=4, stride=2, self_attention=(i==0))]
    nf *= 2
  layers += [
             _conv(nf, nf, ks=3, stride=1),
             _conv(nf, 1, ks=4, bias=False, padding=0, use_activ=False),
             Flatten()]
  return nn.Sequential(*layers)

In [0]:
#AdaptiveLoss matches the target and output size
#Accuracy_thres_expand calculates accuracy, by default binary classification
def colorize_crit_learner(data:ImageDataBunch, loss_critic=AdaptiveLoss(nn.BCEWithLogitsLoss()), nf:int=256):
  return Learner(data, custom_gan_critic(nf=nf), metrics=accuracy_thresh_expand, loss_func=loss_critic, wd=1e-3)